<a href="https://colab.research.google.com/github/mozzaquatro/financial_market/blob/main/Gr%C3%A1fico%20Setores%20Ibov.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q kora

     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 


In [ ]:
!pip install -U plotly

     |████████████████████████████████| 25.3 MB 2.1 MB/s 
  Attempting uninstall: plotly
    Found existing installation: plotly 4.4.1
    Uninstalling plotly-4.4.1:
      Successfully uninstalled plotly-4.4.1


In [ ]:
from kora.selenium import wd

import pandas as pd
import numpy as np
from google.colab.data_table import DataTable
from time import sleep

import plotly.express as px

In [ ]:
def busca_carteira_teorica(indice, espera=8):
  url = f'https://sistemaswebb3-listados.b3.com.br/indexPage/day/{indice.upper()}?language=pt-br'
  wd.get(url)
  sleep(espera)

  wd.find_element_by_id('segment').send_keys("Setor de Atuação")
  sleep(espera)

  wd.find_element_by_link_text("Download").click()
  sleep(espera)

  arquivos = !ls -lt *.csv
  return pd.read_csv(arquivos[0],sep=';', encoding='ISO-8859-1', skipfooter=2, engine='python', thousands='.', decimal=',', header=1, index_col=False )


In [ ]:
ibov = busca_carteira_teorica('ibov',20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning:

find_element_by_* commands are deprecated. Please use find_element() instead

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning:

find_element_by_* commands are deprecated. Please use find_element() instead



FileNotFoundError: ignored

In [ ]:
# ls -lt com erro, engessando o arquivo
ibov = pd.read_csv('IBOVDia_09-12-21.csv',sep=';', encoding='ISO-8859-1', skipfooter=2, engine='python', thousands='.', decimal=',', header=1, index_col=False )

In [ ]:
ibov

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.
0,Bens Indls / Máqs e Equips,WEGE3,WEG,ON NM,1484859030,2.622,2.622
1,Bens Indls / Mat Transporte,EMBR3,EMBRAER,ON NM,734558205,0.767,0.767
2,Bens Indls/Transporte,AZUL4,AZUL,PN N2,327741172,0.421,2.472
3,Bens Indls/Transporte,CCRO3,CCR SA,ON ED NM,1115693556,0.695,2.472
4,Bens Indls/Transporte,ECOR3,ECORODOVIAS,ON NM,339237914,0.149,2.472
...,...,...,...,...,...,...,...
87,Utilidade Públ / Energ Elétrica,ENGI11,ENERGISA,UNT N2,248542219,0.558,6.302
88,Utilidade Públ / Energ Elétrica,ENEV3,ENEVA,ON NM,1261568398,0.925,6.302
89,Utilidade Públ / Energ Elétrica,EGIE3,ENGIE BRASIL,ON NM,255217329,0.491,6.302
90,Utilidade Públ / Energ Elétrica,EQTL3,EQUATORIAL,ON NM,1010511085,1.169,6.302


In [ ]:
ibov['SubSetor'] = ibov['Setor'].apply(lambda s: s[s.rfind("/")+1:].strip())

In [ ]:
ibov['Setor'] = ibov['Setor'].apply(lambda s: s[:s.rfind("/")].strip())

In [ ]:
ibov

,Setor,Código,Ação,Tipo,Qtde. Teórica,Part. (%),Part. (%)Acum.,SubSetor
0,Bens Indls,WEGE3,WEG,ON NM,1484859030,2.622,2.622,Máqs e Equips
1,Bens Indls,EMBR3,EMBRAER,ON NM,734558205,0.767,0.767,Mat Transporte
2,Bens Indls,AZUL4,AZUL,PN N2,327741172,0.421,2.472,Transporte
3,Bens Indls,CCRO3,CCR SA,ON ED NM,1115693556,0.695,2.472,Transporte
4,Bens Indls,ECOR3,ECORODOVIAS,ON NM,339237914,0.149,2.472,Transporte
...,...,...,...,...,...,...,...,...
87,Utilidade Públ,ENGI11,ENERGISA,UNT N2,248542219,0.558,6.302,Energ Elétrica
88,Utilidade Públ,ENEV3,ENEVA,ON NM,1261568398,0.925,6.302,Energ Elétrica
89,Utilidade Públ,EGIE3,ENGIE BRASIL,ON NM,255217329,0.491,6.302,Energ Elétrica
90,Utilidade Públ,EQTL3,EQUATORIAL,ON NM,1010511085,1.169,6.302,Energ Elétrica


In [ ]:
# ver se tem alguma inconsistencia no setores
ibov['Setor'].unique()

array(['Bens Indls', 'Cons N  Básico', 'Cons N Cíclico',
       'Consumo Cíclico', 'Diverso', 'Financ e Outros',
       'Financeiro e Outros', 'Mats Básicos', 'Petróleo', 'Saúde',
       'Tec.Informação', 'Telecomunicaçã', 'Utilidade Públ'], dtype=object)

In [ ]:
# retirando duplicidades
def conserta_setores(setor):
  if setor == 'Cons N  Básico' or setor == 'Cons N Cíclico': return 'Consumo Não-Cíclico'
  if setor == 'Financeiro e Outros' or setor == 'Financ e Outros': return 'Financeiro'
  else: return setor
  


In [ ]:
# aplica funçao conserta setores na coluna
ibov['Setor'] = ibov['Setor'].apply(conserta_setores)

In [ ]:
ibov['Setor'].unique()

array(['Bens Indls', 'Consumo Não-Cíclico', 'Consumo Cíclico', 'Diverso',
       'Financeiro', 'Mats Básicos', 'Petróleo', 'Saúde',
       'Tec.Informação', 'Telecomunicaçã', 'Utilidade Públ'], dtype=object)

In [ ]:
#grafico
fig = px.sunburst(data_frame=ibov,path=['Setor','SubSetor','Código'], values='Part. (%)', height=900)

fig.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:f}%')

fig.show()

In [37]:
#grafico
fig = px.treemap(data_frame=ibov,path=['Setor','SubSetor','Código'], values='Part. (%)', height=900)

fig.update_traces(textfont_color='white',
                  textfont_size=14,
                  hovertemplate='<b>%{label}:</b> %{value:f}%')

fig.show()